<a href="https://colab.research.google.com/github/liangchow/zindi-amazon-secret-runway/blob/main/Data_Visualization/explore_sentinel_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with github: [A guide from Google](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

### Imports and Setup.

In [1]:
%%capture
!pip -q install geojson
!pip -q install rasterio
!pip -q install eeconvert
!pip -q install geemap==0.17.3

In [2]:
# Standard imports
import os
import json

# Geospatial processing packages
import geopandas as gpd
import geojson
import shapely
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
from shapely.geometry import box
from shapely.geometry import Polygon
from pyproj import Transformer

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import geemap.eefolium as emap
import folium

### Mount your Google Drive and install project files

First, we'll mount your Google Drive as we'll save the GeoTiff files to our drive. Then we'll clone the main branch from the GitHub repo so we have access to all of the files from there.
If you want to save your files and make a pull request to github, make a new branch and check out the link at the top.

In [3]:
# mount your drive in case you have any new data uploaded there you want to use
from google.colab import drive
drive.mount('/content/drive')

# clone the main branch from GitHub to get all the data and files from there onto the current runtime session
!apt-get install git
!git clone https://github.com/liangchow/zindi-amazon-secret-runway.git
!git pull # pulls the latest changes from repo

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'zindi-amazon-secret-runway'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 106 (delta 31), reused 7 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 574.34 KiB | 4.10 MiB/s, done.
Resolving deltas: 100% (31/31), done.
fatal: not a git repository (or any of the parent directories): .git


### Setup the main paths to the project files.

In [4]:
airstrip_training_path = '/content/zindi-amazon-secret-runway/Data_Visualization/data/pac_2024_training/pac_2024_training.shp'
base_aoi_path = '/content/zindi-amazon-secret-runway/Data_Visualization//data/shp_test_AOIs'
illegal_runways_path = '/content/zindi-amazon-secret-runway/Data_Visualization/data/nyt_intercept_illegal_runways/Illegal-Airstrips-NYT-Intercept-Public.geojson'

### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine. You will need to edit the following code cell to use your own account. All the Sentinel data will be downloaded through the Google Earth Engine.

In [5]:
ee.Authenticate()
ee.Initialize(project="ee-fortinpascal")

### Let's look at the Sentinel data
We'll use the airstrip training shape file. Each record contains columns for id, year of detection, largo (length probably in meters), Activo (activity status???), and geometry of type *Linestring* with coordinates (LNG LAT) which define points along the airstrip. For each airstrip, we'll define a square AOI to define a boundary when downloading Sentinel data.

In [6]:
# Read the shapefile
airstrip_training_gdf = gpd.read_file(airstrip_training_path)

# Display the data
print(airstrip_training_gdf.head().to_markdown(), "\n")
print(airstrip_training_gdf.info(), "\n")

|    |   id |   yr |    largo |   Activo | geometry                                                                                                                            |
|---:|-----:|-----:|---------:|---------:|:------------------------------------------------------------------------------------------------------------------------------------|
|  0 |    1 | 2023 |  968.918 |        0 | LINESTRING (-70.08928656863503 -13.129844039931504, -70.08052787039317 -13.128251134052485, -70.08052787039317 -13.128251134052485) |
|  1 |    2 | 2022 | 1105.49  |        0 | LINESTRING (-69.16744237255283 -13.620679758207931, -69.1728593987747 -13.612261251288524)                                          |
|  2 |    3 | 2015 |  985.018 |        0 | LINESTRING (-69.14224792429687 -13.694510447986984, -69.14189616934739 -13.685654549844925)                                         |
|  3 |    4 | 2020 |  681.914 |        0 | LINESTRING (-69.07843632939438 -13.787902019520923, -69.08404112675193 -

In [7]:
def create_square_pentagon(center_lon, center_lat, side_length_meters):
  """
  Creates a square geopandas pentagon centered on
  a set of Lon/Lat coordinates.

  Args:
    center_lon: The longitude of the center point.
    center_lat: The latitude of the center point.
    side_length_meters: The length of each side of
                         the square in meters.

  Returns:
    A geopandas GeoDataFrame containing the pentagon.
  """

  # Define the source and destination coordinate systems
  source_crs = "EPSG:4326"  # WGS 84 (Lon/Lat)
  target_crs = "EPSG:3395"  # World Mercator (meters)


  # Create a transformer object to convert between coordinate systems
  transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

  # Transform the center coordinates to Web Mercator
  center_x, center_y = transformer.transform(center_lon, center_lat)

  # Calculate half the side length
  half_side = side_length_meters / 2

  # Calculate the coordinates of the pentagon's vertices in meters
  #coordinates = [
  #    (center_x, center_y + half_side),  # Top
  #    (center_x + half_side, center_y),  # Right
  #    (center_x, center_y - half_side),  # Bottom
  #    (center_x - half_side, center_y),  # Left
  #    (center_x, center_y + half_side)   # Top (close the polygon)
  #]
  coordinates = [
      (center_x - half_side, center_y + half_side),  # Top left
      (center_x + half_side, center_y + half_side),  # Top right
      (center_x + half_side, center_y - half_side),  # Bottom right
      (center_x - half_side, center_y - half_side),  # Bottom Left
      (center_x - half_side, center_y + half_side)   # Top left (close the polygon)
  ]


  # Create a Polygon object from the coordinates
  polygon = Polygon(coordinates)

  # Create a GeoDataFrame from the Polygon with World Mercator CRS
  gdf = gpd.GeoDataFrame(geometry=[polygon], crs=target_crs)

  # Transform the GeoDataFrame back to WGS 84
  gdf = gdf.to_crs(source_crs)

  return gdf

In [8]:
# Temporarily reproject to a projected CRS for accurate centroid calculation
projected_airstrip_training_gdf = airstrip_training_gdf.to_crs(epsg=3395)  # World Mercator projection

# Access the first airstip and calculate its centroid
centroid = projected_airstrip_training_gdf.geometry[0].centroid

# Convert the centroid back to WGS84 (LNG, LAT)
centroid_wgs84 = gpd.GeoSeries([centroid], crs=projected_airstrip_training_gdf.crs).to_crs(epsg=4326).geometry[0]

# Create a new AOI center on the centroid. The lenght of each side is 5km.
newAOI = create_square_pentagon(centroid_wgs84.x, centroid_wgs84.y, 5000)
print(newAOI.info(), "\n")

#fig, ax = plt.subplots(1, figsize=(10,10))
#newAOI.plot(legend=True, ax=ax);

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  1 non-null      geometry
dtypes: geometry(1)
memory usage: 136.0 bytes
None 



**Generate the Sentinel images**

In [18]:
# Select export folder on Google Drive
export_folder = 'Colab Notebooks'

# Select bands (replace with desired bands)
s2_bands = ['B4', 'B3', 'B2', 'B8', 'B11', 'B12']
s1_bands = ['VV', 'VH']


In [19]:
# Function to export an image
def export_image(image,filename_prefix,bands,export_folder):
  date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
  filename = f"{filename_prefix}_{date}"

  task = ee.batch.Export.image.toDrive(
      image=image.select(bands).clip(region),
      description='filename',
      folder=export_folder,
      fileNamePrefix=filename,
      scale=10,  # Adjust scale as needed (meters)
      region=region.getInfo()['coordinates'],
      maxPixels=1e13,
      fileFormat='GeoTIFF'
  )
  task.start()
  print(f'Task ID: {task.id}')
  print('Exporting to Google Drive...')


In [20]:
def download_s2_images(
    region,
    bands=['B4', 'B3', 'B2', 'B8', 'B11', 'B12'],
    filename_prefix='Sentinel2',
    export_folder='Colab Notebooks',
    product='COPERNICUS/S2_HARMONIZED',
    min_date='2023-01-01',
    max_date='2024-06-01',
    range_min=0,
    range_max=2000,
    cloud_pct=10
):

    """Generates cloud-filtered, Sentinel-2 images from
    Google Earth Engine using the Python Earth Engine API.

    Args:
      region (ee.Geometry): The geometry of the area of interest to filter to.
      product (str): Earth Engine asset ID
        You can find the full list of ImageCollection IDs
        at https://developers.google.com/earth-engine/datasets
      min_date (str): Minimum date to acquire collection of satellite images
      max_date (str): Maximum date to acquire collection of satellite images
      range_min (int): Minimum value for visalization range
      range_max (int): Maximum value for visualization range
      cloud_pct (float): The cloud cover percent to filter by (default 10)

    Returns:
      ee.image.Image: Generated Sentinel-2 images clipped to the region of interest
    """

    # Generate collection of images
    sentinel2 = ee.ImageCollection(product)\
        .filterBounds(region)\
        .filterDate(str(min_date), str(max_date))\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_pct))

    # Apply the export function to each image in the collection.
    collection_list = sentinel2.toList(sentinel2.size())
    for i in range(sentinel2.size().getInfo()):
      image = ee.Image(collection_list.get(i))
      export_image(image=image,filename_prefix=filename_prefix,bands=bands,export_folder=export_folder)

    print("Export tasks should be complete.")


In [24]:
def download_s1_images(
    region,
    bands = ['VV', 'VH'],
    filename_prefix='Sentinel2',
    export_folder='Colab Notebooks',
    product='COPERNICUS/S1_GRD',
    min_date='2024-04-01',
    max_date='2024-06-01',
):

    """Generates Sentinel-1 images from Google Earth Engine using the Python Earth Engine API.

    Args:
      region (ee.Geometry): The geometry of the area of interest to filter to.
      bands: List of bands
      filename_prefix: Filename prexix to save to disk
      export_folder: Google Drive folder to save to
      product (str): Earth Engine asset ID
        You can find the full list of ImageCollection IDs
        at https://developers.google.com/earth-engine/datasets
      min_date (str): Minimum date to acquire collection of satellite images
      max_date (str): Maximum date to acquire collection of satellite images
      range_min (int): Minimum value for visalization range
      range_max (int): Maximum value for visualization range
      cloud_pct (float): The cloud cover percent to filter by (default 10)

    Returns:
      ee.image.Image: Generated Sentinel-1 images clipped to the region of interest
    """

    # Define the Sentinel-1 image collection and filter by date and region.
    sentinel1 = ee.ImageCollection(product) \
        .filterDate(min_date, max_date) \
        .filterBounds(region) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide (IW) swath mode

    # Apply the export function to each image in the collection.
    collection_list = sentinel1.toList(sentinel1.size())
    for i in range(sentinel1.size().getInfo()):
      image = ee.Image(collection_list.get(i))
      export_image(image=image,filename_prefix=filename_prefix,bands=bands,export_folder=export_folder)

    print("Export tasks should be complete.")


In [56]:
def plot_s1_images(
    region,
    bands = ['VV', 'VH'],
    product='COPERNICUS/S1_GRD',
    min_date='2023-06-01',
    max_date='2023-07-01',
):

    """Generates Sentinel-1 images from Google Earth Engine using the Python Earth Engine API.

    Args:
      region (ee.Geometry): The geometry of the area of interest to filter to.
      bands: List of bands
      product (str): Earth Engine asset ID
        You can find the full list of ImageCollection IDs
        at https://developers.google.com/earth-engine/datasets
      min_date (str): Minimum date to acquire collection of satellite images
      max_date (str): Maximum date to acquire collection of satellite images

    Returns:
      Map
    """

    # Define the Sentinel-1 image collection and filter by date and region.
    sentinel1 = ee.ImageCollection(product) \
        .filterDate(min_date, max_date) \
        .filterBounds(region) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide (IW) swath mode

    # Get the most recent image in the collection.
    most_recent_image = sentinel1.sort('system:time_start', False).first().clip(region)

    # Select the VV and VH bands.
    vv_vh_image = most_recent_image.select(['VV', 'VH'])

    # Create a map.
    location = region.centroid().coordinates().getInfo()[::-1]
    Map = geemap.Map(center=location, zoom=10, basemap='SATELLITE')

    # Add Google TERRAIN as a base map.
    Map.add_basemap('TERRAIN')

    # Define visualization parameters for the VV and VH bands.
    #vv_vis_params = {
    #    'min': -25,
    #    'max': 0,
    #    'palette': ['blue', 'white', 'green']
    #}
    vv_vis_params = {
        'min': -25,
        'max': 0,
        'palette': ['black', 'white']
    }


    vh_vis_params = {
        'min': -25,
        'max': 0,
        'palette': ['purple', 'white', 'orange']
    }

    # Make an RGB color composite image (VV,VH,VV/VH).
    rgb = ee.Image.rgb(vv_vh_image.select('VV'),
                       vv_vh_image.select('VH'),
                       vv_vh_image.select('VV').divide(vv_vh_image.select('VH')))


    # Add the S1 rgb composite to the map object.
    Map.addLayer(rgb, {'min': [-25, -25, 0], 'max': [0, 0, 2]}, 'Fake S1 RGB')

    # Add the VV band layer to the map.
    Map.addLayer(vv_vh_image.select('VV'), vv_vis_params, 'VV Band')

    # Add the VH band layer to the map.
    Map.addLayer(vv_vh_image.select('VH'), vh_vis_params, 'VH Band')

    # Add the region of interest as a layer.
    Map.addLayer(region, {}, 'Region of Interest')

    # Display the map.
    Map.addLayerControl()  # This line will add a layer control to toggle the layers on/off.
    return Map


In [57]:
newAOI_fc = geemap.geopandas_to_ee(newAOI)
region = newAOI_fc.geometry()
#download_s1_images(region, bands=s1_bands, filename_prefix='Sentinel1', export_folder=export_folder)
#download_s2_images(region, bands=s2_bands, filename_prefix='Sentinel2', export_folder=export_folder)
Map = plot_s1_images(region, bands=s1_bands)
Map

Map(center=[-13.12904690589121, -70.0849072195116], controls=(WidgetControl(options=['position', 'transparent_…